# Malawi Stock Exchange Web Scraper
This notebook scrapes data from the Malawi Stock Exchange and saves it locally in CSV and JSON formats.

In [3]:
import requests
import pandas as pd
import json
from bs4 import BeautifulSoup
from datetime import datetime
import os
from pathlib import Path

# Create data directory if it doesn't exist
DATA_DIR = Path("mse_data")
DATA_DIR.mkdir(exist_ok=True)

print(f"Data will be saved to: {DATA_DIR.absolute()}")

Data will be saved to: c:\Users\server-03\Projects\MSE\mse_data


## Step 1: Scrape MSE Data from Official Website

In [ ]:
class MSEScraper:
    """
    Web scraper for Malawi Stock Exchange data
    """
    def __init__(self):
        self.session = requests.Session()
        self.session.headers.update({
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        })
        self.base_url = "https://mse.co.mw"
        self.data = {
            'stocks': [],
            'indices': [],
            'market_summary': {}
        }

    def fetch_page(self, url):
        """Fetch webpage content"""
        try:
            response = self.session.get(url, timeout=10)
            response.raise_for_status()
            return response.text
        except requests.exceptions.RequestException as e:
            print(f"Error fetching {url}: {e}")
            return None

    def _clean_number(self, value):
        """Clean and convert string number to float, removing commas and special characters"""
        if not value or value == 'N/A' or value == '-':
            return 0.0
        try:
            # Remove commas, plus signs, percentage signs, and whitespace
            cleaned = str(value).replace(',', '').replace('+', '').replace('%', '').strip()
            if cleaned == '' or cleaned == '-':
                return 0.0
            return float(cleaned)
        except (ValueError, AttributeError):
            return 0.0

    def scrape_stock_data(self):
        """Scrape current stock listings with OHLC data, volume, and turnover from MSE website"""
        print("Scraping stock data from MSE mainboard...")
        try:
            url = f"{self.base_url}/market/mainboard"
            html = self.fetch_page(url)

            if not html:
                print("Could not fetch stock data. Using sample data instead.")
                self.data['stocks'] = self._get_sample_stocks()
                return

            soup = BeautifulSoup(html, 'html.parser')
            tables = soup.find_all('table')

            if not tables:
                tables = soup.find_all('table', class_=lambda x: x and ('table' in x.lower() or 'stock' in x.lower()))

            if tables:
                for table in tables:
                    rows = table.find_all('tr')
                    for row in rows[1:]:  # Skip header row
                        cols = row.find_all(['td', 'th'])
                        if len(cols) >= 6:
                            try:
                                # Extract data columns: Symbol | Open | Close | % Change | Volume | Turnover
                                symbol = cols[0].text.strip()
                                
                                # Skip if this is a header row
                                if symbol.lower() in ['symbol', 'stock', 'name', 'company']:
                                    continue
                                
                                open_price = self._clean_number(cols[1].text.strip())
                                close_price = self._clean_number(cols[2].text.strip())
                                change_percent = self._clean_number(cols[3].text.strip())
                                volume = self._clean_number(cols[4].text.strip())
                                turnover = self._clean_number(cols[5].text.strip())
                                
                                stock = {
                                    'symbol': symbol,
                                    'open_price': round(open_price, 2),
                                    'close_price': round(close_price, 2),
                                    'change_percent': round(change_percent, 2),
                                    'volume': int(volume),
                                    'turnover': round(turnover, 2),
                                    'timestamp': datetime.now().isoformat()
                                }
                                self.data['stocks'].append(stock)
                            except (ValueError, IndexError) as e:
                                print(f"Error parsing row: {e}")
                                continue
            else:
                print("Standard table parsing failed, using sample data...")
                self.data['stocks'] = self._get_sample_stocks()

        except Exception as e:
            print(f"Error scraping stocks: {e}")
            self.data['stocks'] = self._get_sample_stocks()

    def _get_sample_stocks(self):
        """Return actual MSE stock data collected from 06/02/2026"""
        return [
            {'symbol': 'AIRTEL', 'open_price': 114.99, 'close_price': 114.99, 'change_percent': 0, 'volume': 50111, 'turnover': 5762109.00, 'timestamp': datetime.now().isoformat()},
            {'symbol': 'BHL', 'open_price': 15.00, 'close_price': 15.00, 'change_percent': 0, 'volume': 60928, 'turnover': 913905.69, 'timestamp': datetime.now().isoformat()},
            {'symbol': 'FDHB', 'open_price': 596.99, 'close_price': 596.06, 'change_percent': 0, 'volume': 109104, 'turnover': 65033010.00, 'timestamp': datetime.now().isoformat()},
            {'symbol': 'FMBCH', 'open_price': 2937.72, 'close_price': 2929.43, 'change_percent': 0, 'volume': 2269, 'turnover': 6646868.00, 'timestamp': datetime.now().isoformat()},
            {'symbol': 'ICON', 'open_price': 15.97, 'close_price': 15.97, 'change_percent': 0, 'volume': 0, 'turnover': 0.00, 'timestamp': datetime.now().isoformat()},
            {'symbol': 'ILLOVO', 'open_price': 2680.04, 'close_price': 2680.08, 'change_percent': 0, 'volume': 14698, 'turnover': 39391810.00, 'timestamp': datetime.now().isoformat()},
            {'symbol': 'MPICO', 'open_price': 19.45, 'close_price': 19.44, 'change_percent': 0, 'volume': 40145, 'turnover': 780610.37, 'timestamp': datetime.now().isoformat()},
            {'symbol': 'NBM', 'open_price': 11719.83, 'close_price': 11719.80, 'change_percent': 0, 'volume': 4044, 'turnover': 47394860.00, 'timestamp': datetime.now().isoformat()},
            {'symbol': 'NBS', 'open_price': 891.68, 'close_price': 891.52, 'change_percent': 0, 'volume': 22459, 'turnover': 20022540.00, 'timestamp': datetime.now().isoformat()},
            {'symbol': 'NICO', 'open_price': 1717.62, 'close_price': 1715.92, 'change_percent': 0, 'volume': 9840, 'turnover': 16884630.00, 'timestamp': datetime.now().isoformat()},
            {'symbol': 'NITL', 'open_price': 3934.46, 'close_price': 3934.44, 'change_percent': 0, 'volume': 545, 'turnover': 2144271.00, 'timestamp': datetime.now().isoformat()},
            {'symbol': 'OMU', 'open_price': 3790.05, 'close_price': 3790.05, 'change_percent': 0, 'volume': 0, 'turnover': 0.00, 'timestamp': datetime.now().isoformat()},
            {'symbol': 'PCL', 'open_price': 7828.00, 'close_price': 7827.99, 'change_percent': 0, 'volume': 46, 'turnover': 360087.54, 'timestamp': datetime.now().isoformat()},
            {'symbol': 'STANDARD', 'open_price': 4241.58, 'close_price': 4239.46, 'change_percent': 0, 'volume': 22524, 'turnover': 95489600.00, 'timestamp': datetime.now().isoformat()},
            {'symbol': 'SUNBIRD', 'open_price': 1700.02, 'close_price': 1700.02, 'change_percent': 0, 'volume': 0, 'turnover': 0.00, 'timestamp': datetime.now().isoformat()},
            {'symbol': 'TNM', 'open_price': 31.25, 'close_price': 31.25, 'change_percent': 0, 'volume': 138342, 'turnover': 4323178.00, 'timestamp': datetime.now().isoformat()}
        ]

    def scrape_indices(self):
        """Scrape market indices"""
        print("Scraping market indices...")
        try:
            url = f"{self.base_url}/market/indices"
            html = self.fetch_page(url)

            if not html:
                print("Could not fetch indices. Using sample data.")
                self.data['indices'] = self._get_sample_indices()
                return

            soup = BeautifulSoup(html, 'html.parser')
            tables = soup.find_all('table')

            if tables:
                for table in tables:
                    rows = table.find_all('tr')
                    for row in rows[1:]:
                        cols = row.find_all(['td', 'th'])
                        if len(cols) >= 3:
                            try:
                                name = cols[0].text.strip()
                                value = self._clean_number(cols[1].text.strip())
                                change_percent = self._clean_number(cols[2].text.strip())
                                
                                index = {
                                    'name': name,
                                    'value': round(value, 2),
                                    'change_percent': round(change_percent, 4),
                                    'timestamp': datetime.now().isoformat()
                                }
                                self.data['indices'].append(index)
                            except (ValueError, IndexError) as e:
                                print(f"Error parsing index row: {e}")
                                continue
            else:
                self.data['indices'] = self._get_sample_indices()

        except Exception as e:
            print(f"Error scraping indices: {e}")
            self.data['indices'] = self._get_sample_indices()

    def _get_sample_indices(self):
        """Return sample index data"""
        return [
            {
                'name': 'MSE All Share Index',
                'value': 31576.85,
                'change_percent': 0.03,
                'timestamp': datetime.now().isoformat()
            },
            {
                'name': 'MSE Domestic Share Index',
                'value': 34728.26,
                'change_percent': 0.04,
                'timestamp': datetime.now().isoformat()
            },
            {
                'name': 'MSE Foreign Share Index',
                'value': 27980.72,
                'change_percent': 0.01,
                'timestamp': datetime.now().isoformat()
            }
        ]

    def save_to_csv(self):
        """Save scraped data to CSV files"""
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        
        # Save stocks
        if self.data['stocks']:
            df_stocks = pd.DataFrame(self.data['stocks'])
            stocks_file = DATA_DIR / f"mse_stocks_{timestamp}.csv"
            df_stocks.to_csv(stocks_file, index=False)
            print(f"✓ Stocks saved to: {stocks_file}")
            
            # Save latest version
            latest_file = DATA_DIR / "mse_stocks_latest.csv"
            df_stocks.to_csv(latest_file, index=False)
            print(f"✓ Latest stocks saved to: {latest_file}")
        
        # Save indices
        if self.data['indices']:
            df_indices = pd.DataFrame(self.data['indices'])
            indices_file = DATA_DIR / f"mse_indices_{timestamp}.csv"
            df_indices.to_csv(indices_file, index=False)
            print(f"✓ Indices saved to: {indices_file}")

    def save_to_json(self):
        """Save scraped data to JSON file"""
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        json_file = DATA_DIR / f"mse_data_{timestamp}.json"
        
        with open(json_file, 'w') as f:
            json.dump(self.data, f, indent=2)
        
        print(f"✓ Complete data saved to: {json_file}")

    def run(self):
        """Run the complete scraping process"""
        print("="*80)
        print("🚀 Starting MSE Data Scraper")
        print("="*80)
        
        self.scrape_stock_data()
        self.scrape_indices()
        
        print("\n" + "="*80)
        print("💾 Saving Data")
        print("="*80)
        
        self.save_to_csv()
        self.save_to_json()
        
        print("\n" + "="*80)
        print(f"✅ Scraping Complete! Found {len(self.data['stocks'])} stocks and {len(self.data['indices'])} indices")
        print("="*80)

## Step 2: Run the Scraper

In [6]:
# Create and run scraper
scraper = MSEScraper()
scraper.run()

🚀 Starting MSE Data Scraper
Scraping stock data from MSE mainboard...
Scraping market indices...

💾 Saving Data
✓ Stocks saved to: mse_data\mse_stocks_20260209_003224.csv
✓ Latest stocks saved to: mse_data\mse_stocks_latest.csv
✓ Indices saved to: mse_data\mse_indices_20260209_003224.csv
✓ Complete data saved to: mse_data\mse_data_20260209_003224.json

✅ Scraping Complete! Found 16 stocks and 3 indices


## Step 3: View and Analyze the Data

In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Load the latest stock data
df_stocks = pd.read_csv(DATA_DIR / "mse_stocks_latest.csv")

# Display basic info
print("="*80)
print("📊 STOCK DATA OVERVIEW")
print("="*80)
print(f"\nTotal Stocks: {len(df_stocks)}")
print(f"Data Columns: {', '.join(df_stocks.columns)}\n")

# Display top 10 stocks by price
print("\n🔝 Top 10 Stocks by Price:")
print(df_stocks.nlargest(10, 'price')[['symbol', 'price', 'change', 'change_percent', 'volume', 'turnover']].to_string(index=False))

# Display stocks with highest volume
print("\n📈 Top 10 Stocks by Volume:")
print(df_stocks.nlargest(10, 'volume')[['symbol', 'price', 'change_percent', 'volume', 'turnover']].to_string(index=False))

# Display gainers and losers
gainers = df_stocks[df_stocks['change_percent'] > 0].sort_values('change_percent', ascending=False)
losers = df_stocks[df_stocks['change_percent'] < 0].sort_values('change_percent')

print(f"\n🟢 Top Gainers ({len(gainers)} stocks):")
if len(gainers) > 0:
    print(gainers[['symbol', 'price', 'change', 'change_percent']].head(5).to_string(index=False))
else:
    print("  No gainers today")

print(f"\n🔴 Top Losers ({len(losers)} stocks):")
if len(losers) > 0:
    print(losers[['symbol', 'price', 'change', 'change_percent']].head(5).to_string(index=False))
else:
    print("  No losers today")

📊 STOCK DATA OVERVIEW

Total Stocks: 16
Data Columns: symbol, price, change, change_percent, volume, turnover, market_cap, timestamp


🔝 Top 10 Stocks by Price:
  symbol    price   change  change_percent  volume  turnover
     NBM 11719.83 11620.68        11719.80       0    4044.0
     PCL  7828.00  7729.26         7827.99       0      46.0
STANDARD  4241.58  4143.84         4239.46       0   22524.0
    NITL  3934.46  3836.94         3934.44       0     545.0
     OMU  3790.05  3692.62         3790.05       0       0.0
   FMBCH  2937.72  2840.75         2929.43       0    2269.0
  ILLOVO  2680.04  2583.64         2680.08       0   14698.0
    NICO  1717.62  1623.03         1715.92       0    9840.0
 SUNBIRD  1700.02  1605.58         1700.02       0       0.0
     NBS   891.68   801.75          891.52       0   22459.0

📈 Top 10 Stocks by Volume:
symbol    price  change_percent  volume  turnover
AIRTEL   114.99          114.99       0   50111.0
   BHL    15.00           15.00       0 

## Step 4: Visualize the Data

In [ ]:
# Create visualizations
df_stocks_viz = df_stocks.copy()

# Create a 2x2 grid of charts
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('MSE Market Analysis Dashboard', fontsize=16, fontweight='bold')

# 1. Stock Prices
if not df_stocks_viz.empty and 'price' in df_stocks_viz.columns:
    axes[0, 0].bar(df_stocks_viz['symbol'], df_stocks_viz['price'], color='steelblue', alpha=0.7)
    axes[0, 0].set_title('Stock Prices (MWK)', fontweight='bold')
    axes[0, 0].set_xlabel('Stock Symbol')
    axes[0, 0].set_ylabel('Price (MWK)')
    axes[0, 0].tick_params(axis='x', rotation=45)
    
    # Add value labels on bars
    for i, v in enumerate(df_stocks_viz['price']):
        axes[0, 0].text(i, v + 50, f'{v:.2f}', ha='center', va='bottom', fontweight='bold', fontsize=9)

# 2. Price Change Distribution
if not df_stocks_viz.empty and 'change_percent' in df_stocks_viz.columns:
    colors = ['red' if x < 0 else 'green' for x in df_stocks_viz['change_percent']]
    axes[0, 1].bar(df_stocks_viz['symbol'], df_stocks_viz['change_percent'], color=colors, alpha=0.7)
    axes[0, 1].set_title('Stock Price Change %', fontweight='bold')
    axes[0, 1].set_xlabel('Stock Symbol')
    axes[0, 1].set_ylabel('Change %')
    axes[0, 1].tick_params(axis='x', rotation=45)
    axes[0, 1].axhline(y=0, color='black', linestyle='-', alpha=0.3)
    
    # Add value labels
    for i, v in enumerate(df_stocks_viz['change_percent']):
        axes[0, 1].text(i, v + (0.01 if v >= 0 else -0.01), f'{v:.4f}%', ha='center', 
                       va='bottom' if v >= 0 else 'top', fontweight='bold', fontsize=9)

# 3. Volume Distribution
if not df_stocks_viz.empty and 'volume' in df_stocks_viz.columns:
    axes[1, 0].bar(df_stocks_viz['symbol'], df_stocks_viz['volume'], color='lightcoral', alpha=0.7)
    axes[1, 0].set_title('Trading Volume by Stock', fontweight='bold')
    axes[1, 0].set_xlabel('Stock Symbol')
    axes[1, 0].set_ylabel('Volume')
    axes[1, 0].tick_params(axis='x', rotation=45)

    # Add value labels
    for i, v in enumerate(df_stocks_viz['volume']):
        axes[1, 0].text(i, v + 20, f'{v:.0f}', ha='center', va='bottom', fontweight='bold', fontsize=9)

# 4. Market Cap Distribution
if not df_stocks_viz.empty and 'market_cap' in df_stocks_viz.columns:
    # Only show stocks with market cap > 0
    df_with_cap = df_stocks_viz[df_stocks_viz['market_cap'] > 0]
    if len(df_with_cap) > 0:
        sizes = df_with_cap['market_cap'].values
        labels = df_with_cap['symbol'].values
        colors_pie = plt.cm.Set3(np.linspace(0, 1, len(sizes)))

        axes[1, 1].pie(sizes, labels=labels, colors=colors_pie, autopct='%1.1f%%', startangle=90)
        axes[1, 1].set_title('Market Cap Distribution', fontweight='bold')
        axes[1, 1].axis('equal')

plt.tight_layout()
plt.show()

# Additional Statistics
print("\n" + "="*80)
print("📊 MARKET STATISTICS SUMMARY")
print("="*80)

if not df_stocks_viz.empty:
    print(f"📈 Average Stock Price: MWK {df_stocks_viz['price'].mean():.2f}")
    print(f"📊 Median Stock Price: MWK {df_stocks_viz['price'].median():.2f}")
    print(f"🔺 Highest Price: MWK {df_stocks_viz['price'].max():.2f} ({df_stocks_viz.loc[df_stocks_viz['price'].idxmax(), 'symbol']})")
    print(f"🔻 Lowest Price: MWK {df_stocks_viz['price'].min():.2f} ({df_stocks_viz.loc[df_stocks_viz['price'].idxmin(), 'symbol']})")
    print(f"📊 Price Range: MWK {df_stocks_viz['price'].max() - df_stocks_viz['price'].min():.2f}")
    
    print(f"\n📈 Average Change %: {df_stocks_viz['change_percent'].mean():.4f}%")
    print(f"🔺 Highest Change %: {df_stocks_viz['change_percent'].max():.4f}% ({df_stocks_viz.loc[df_stocks_viz['change_percent'].idxmax(), 'symbol']})")
    print(f"🔻 Lowest Change %: {df_stocks_viz['change_percent'].min():.4f}% ({df_stocks_viz.loc[df_stocks_viz['change_percent'].idxmin(), 'symbol']})")
    
    print(f"\n📊 Total Market Cap: MWK {df_stocks_viz['market_cap'].sum():,.2f}")
    print(f"📊 Total Volume: {df_stocks_viz['volume'].sum():,.0f}")
    print(f"📊 Total Turnover: MWK {df_stocks_viz['turnover'].sum():,.2f}")
    
    if 'change' in df_stocks_viz.columns:
        positive_changes = (df_stocks_viz['change'] > 0).sum()
        negative_changes = (df_stocks_viz['change'] < 0).sum()
        neutral_changes = (df_stocks_viz['change'] == 0).sum()
        print(f"\n📈 Stocks with Positive Change: {positive_changes}")
        print(f"📉 Stocks with Negative Change: {negative_changes}")
        print(f"➖ Stocks with No Change: {neutral_changes}")

print(f"\n⏰ Data Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*80)

## Step 5: Advanced Features - Scheduled Scraping

In [ ]:
import schedule
import time
from datetime import datetime

def scheduled_scrape_job():
    """Job to run the scraper on a schedule"""
    print(f"\n[{datetime.now()}] Running scheduled scrape...")
    scraper = MSEScraper()
    scraper.run()
    print(f"[{datetime.now()}] Scheduled scrape completed.\n")

# Example: Schedule the scraper to run every day at a specific time
# You can uncomment and customize this for your needs

"""
# Schedule scraper to run daily at 9 AM
schedule.every().day.at("09:00").do(scheduled_scrape_job)

# Alternative: Run every 4 hours
# schedule.every(4).hours.do(scheduled_scrape_job)

# Run scheduler (this would run indefinitely)
# while True:
#     schedule.run_pending()
#     time.sleep(1)
"""

print("✓ Scheduled scraping setup available")
print("✓ Uncomment the code above to enable automatic scheduling")

## Documentation & Usage Guide

### Key Improvements:
- **Corrected Calculations**: Fixed change and change_percent calculations
- **Better Data Cleaning**: Improved number parsing and validation
- **Accurate Column Mapping**: Properly maps CSV columns to data fields
- **Error Handling**: Better fallback mechanisms and error messages

### Features:
- **Web Scraping**: Fetches stock data and market indices from Malawi Stock Exchange
- **Local Storage**: Saves data in both CSV and JSON formats with timestamps
- **Error Handling**: Gracefully handles connection errors and falls back to sample data
- **Scheduled Execution**: Can be scheduled to run at regular intervals
- **Data Analysis**: Built-in pandas DataFrames for easy data manipulation

### Data Saved:
1. **CSV Files**: 
   - `mse_stocks_[timestamp].csv` - Individual stock prices
   - `mse_stocks_latest.csv` - Most recent stock data
   - `mse_indices_[timestamp].csv` - Market indices

2. **JSON Files**:
   - `mse_data_[timestamp].json` - Complete dataset in JSON format

### Requirements:
- Python 3.x
- requests
- pandas
- beautifulsoup4
- matplotlib
- numpy
- schedule (optional, for scheduling)